# SC-SINDy Test Runner

This notebook clones the Structure-Constrained SINDy repository and runs the test suite.

**Run this in Google Colab to test the package.**

## 1. Setup: Clone Repository and Install Dependencies

In [ ]:
import os

# Repository configuration
REPO_URL = "https://github.com/yourusername/structure-constrained-sindy.git"  # UPDATE THIS
REPO_DIR = "structure-constrained-sindy"

# Check if repo already exists
if os.path.exists(REPO_DIR):
    print(f"Repository already exists at {REPO_DIR}")
    %cd {REPO_DIR}
    !git pull  # Update to latest
else:
    print(f"Cloning repository...")
    !git clone {REPO_URL}
    %cd {REPO_DIR}

print(f"\nCurrent directory: {os.getcwd()}")

In [ ]:
# Install the package in development mode with all dependencies
!pip install -e ".[dev,torch,viz]" -q
print("Installation complete!")

In [ ]:
# Run speed benchmarks
!python tests/benchmarks/benchmark_speed.py

In [ ]:
# Run accuracy benchmarks
!python tests/benchmarks/benchmark_accuracy.py

## 6. Quick Functionality Test

In [ ]:
# Quick test to verify the package works
import numpy as np
from sc_sindy import (
    sindy_stls,
    build_library_2d,
    compute_derivatives_finite_diff,
    compute_structure_metrics,
    format_equation,
)
from sc_sindy.systems import VanDerPol

print("Testing SC-SINDy on Van der Pol oscillator...")
print("=" * 50)

# Generate data
system = VanDerPol(mu=1.0)
t = np.linspace(0, 20, 2000)
X = system.generate_trajectory(np.array([1.0, 0.0]), t)
X_dot = compute_derivatives_finite_diff(X, t[1] - t[0])

print(f"Generated {len(t)} data points")

# Build library
Theta, labels = build_library_2d(X)
print(f"Built library with {len(labels)} terms: {labels}")

# Run SINDy
xi, elapsed = sindy_stls(Theta, X_dot, threshold=0.1)
print(f"SINDy completed in {elapsed:.4f} seconds")

# Display equations
print("\nDiscovered equations:")
for i, var in enumerate(['x', 'y']):
    eq = format_equation(xi[i], labels)
    print(f"  d{var}/dt = {eq}")

# Evaluate
true_xi = system.get_true_coefficients(labels)
metrics = compute_structure_metrics(xi, true_xi)
print(f"\nStructure recovery metrics:")
print(f"  Precision: {metrics['precision']:.3f}")
print(f"  Recall: {metrics['recall']:.3f}")
print(f"  F1 Score: {metrics['f1']:.3f}")

print("\n" + "=" * 50)
print("All tests passed!")

## 7. Test 3D System (Lorenz)

In [ ]:
from sc_sindy import build_library_3d
from sc_sindy.systems import Lorenz

print("Testing SC-SINDy on Lorenz system...")
print("=" * 50)

# Generate data
system = Lorenz(sigma=10, rho=28, beta=8/3)
t = np.linspace(0, 10, 10000)
X = system.generate_trajectory(np.array([1.0, 1.0, 1.0]), t)

# Subsample
X_sub = X[::10]
t_sub = t[::10]
X_dot = compute_derivatives_finite_diff(X_sub, t_sub[1] - t_sub[0])

print(f"Using {len(t_sub)} data points")

# Build library and run SINDy
Theta, labels = build_library_3d(X_sub)
xi, elapsed = sindy_stls(Theta, X_dot, threshold=0.5)

print(f"Built library with {len(labels)} terms")
print(f"SINDy completed in {elapsed:.4f} seconds")

# Display equations
print("\nDiscovered equations:")
for i, var in enumerate(['x', 'y', 'z']):
    eq = format_equation(xi[i], labels)
    print(f"  d{var}/dt = {eq}")

# Evaluate
true_xi = system.get_true_coefficients(labels)
metrics = compute_structure_metrics(xi, true_xi)
print(f"\nStructure recovery: F1 = {metrics['f1']:.3f}")

## 8. Test with Noisy Data

In [ ]:
print("Testing noise robustness...")
print("=" * 50)

system = VanDerPol(mu=1.0)
t = np.linspace(0, 20, 2000)

noise_levels = [0.0, 0.01, 0.05, 0.10]
results = []

for noise in noise_levels:
    X = system.generate_trajectory(np.array([1.0, 0.0]), t, noise_level=noise)
    X_dot = compute_derivatives_finite_diff(X, t[1] - t[0])
    Theta, labels = build_library_2d(X)
    xi, _ = sindy_stls(Theta, X_dot, threshold=0.1 + noise)
    
    true_xi = system.get_true_coefficients(labels)
    metrics = compute_structure_metrics(xi, true_xi)
    results.append((noise, metrics['f1']))
    print(f"Noise level {noise:.2f}: F1 = {metrics['f1']:.3f}")

print("\nNoise robustness test complete!")

## Summary

This notebook has:
1. Cloned the repository (or updated if already present)
2. Installed all dependencies
3. Run unit tests
4. Run integration tests
5. Run benchmarks
6. Performed quick functionality tests

All tests should pass for a properly configured repository.